In [ ]:
import cProfile
import io
import math
import pstats
import random

import numpy as np
import torch
import socialforce

[Sven Kreiss](https://www.svenkreiss.com), March 4 2021

# Performance

## Fitting to 1, 5 and 20 Circle Scenarios each with Two Pedestrians

The preferred speed needs to be varied. Otherwise the symmetry of the problem creates unrealistic scenarios where the two pedestrians get stuck.

In [ ]:
def generate_gt(n):
    torch.manual_seed(42)
    np.random.seed(42)

    # ped0 always left to right
    ped0 = np.array([-5.0, 0.0, 1.0, 0.0, 5.0, 0.0])

    generator_initial_states = []
    for theta in np.random.rand(n) * 2.0 * math.pi:
        # ped1 at a random angle with +/-20% speed variation
        c, s = np.cos(theta), np.sin(theta)
        r = np.array([[c, -s], [s, c]])
        ped1 = np.concatenate((
            np.matmul(r, ped0[0:2]),
            np.matmul(r, ped0[2:4]) * (0.8 + np.random.rand(1) * 0.4),
            np.matmul(r, ped0[4:6]),
        ))
        generator_initial_states.append(
            np.stack((ped0, ped1))
        )

    generator_ped_ped = socialforce.PedPedPotential(2.1)
    with torch.no_grad():
        trajectories = [
            socialforce.Simulator(initial_state, ped_ped=generator_ped_ped).run(21)
            for initial_state in generator_initial_states
        ]
    return generator_ped_ped, trajectories

generator_pedped_1, scenarios_1 = generate_gt(1)

In [ ]:
true_experience = socialforce.Trainer.scenes_to_experience(scenarios_1)
V = socialforce.PedPedPotentialMLP().double()
initial_parameters = V.state_dict()

def simulator_factory(initial_state):
    return socialforce.Simulator(initial_state, ped_ped=V)

opt = torch.optim.SGD(V.parameters(), lr=3.0)
with cProfile.Profile() as pr:
    socialforce.Trainer(simulator_factory, opt, true_experience, batch_size=1).loop(1)

In [ ]:
ps = pstats.Stats(pr).strip_dirs().sort_stats('tottime')
ps.dump_stats('simulator.prof')
ps.print_stats()

In [ ]:
!flameprof simulator.prof > simulator_flame.svg

In [ ]:
import IPython
IPython.display.SVG(filename='simulator_flame.svg')